# Scottish Election Voting Outcomes
> Scottish local elections operate under a Single Transferable Vote (STV) system. Each voter is free to rank as many candidates as they like. If their first choice drops out of the race due to not reaching the required nr of votes, their second choice receives their vote instead, and so on. In this project, we look to aggregate vote counts from 338 wards across Scotland from a range of file types into one comprehensive cleaned csv.

- toc: false
- branch: master
- badges: false
- comments: true
- categories: [fastpages, jupyter]
- image: images/elections.jpeg
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

## Objectives

Local election outcomes are collected and aggregated at ward level. Each ward or county has variations in how outcomes are recorded, formatted and filed. 

This project demonstrates how object-oriented programming can be used to create ETL pipelines to unify all these files into national-level data. 

We define a class of data transformers to:

- **Extract** text data from 330 files with 5 different types: .csv, .txt, .xls, .xlsx, and .blt
- **Transform** this data through a series of steps detailed below, and 
- **Load** an aggregated .csv file back into the directory.

## File Structures

The 330 files all have similar structures. 
Each row represents a possible voting pattern, i.e. a possible combination of ranking candidates, along with the nr of times that particular combination occured. For example:

`122, 4, 2, 3, 1, 6`

represents that 122 votes were cast with the first preference going to candidate 4. Should that candidate drop out of the race, the vote should go to candidate 2, and so forth.

The second to last row of the file lists the candidates and their party affiliations. The nr in the votes correspond to their ordering. i.e. `1` refers to the first candidate in the list, and the party they belong to.

The last row in the file cites the ward the data corresponds to.

## Transformations

## Class Definitions

In [1]:
import pandas as pd
import glob
import os, os.path


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def create_object(path):
    
    # Dictionary pointing to the correct Class for each file extension
    object_map = {
        'csv' : Ward_Data_csv,
        'xls': Ward_Data_csv,
        'txt' : Ward_Data_txt,
        'blt' : Ward_Data_txt,
        'xlsx': Ward_Data_xlsx
        
    }
    
    # Determine the file extension
    extension = path[2:].split('.')[1]
    assert extension in object_map, 'Invalid file type. Must be csv, xls, xlsx, txt or blt'
    
    # return the corresponding object 
    return object_map[extension](path)
        
            

In [3]:
from sympy import subsets


class Ward_Data():
    
    '''
    The default assumed filetype for the superclass is .csv
    Generally, the correct subclass should be used instead:
    Ward_Data_csv, Ward_Data_txt, Ward_data_xlsx
    '''
    
    def __init__(self, path):
        self.path = path
        return
    
    def read_data(self):
        # Check if top row contains text
        
        # Read in data
        self.data = pd.read_csv(self.path, header = None, delimiter = ';')
        #print(self.data.shape)
        
        assert self.data.shape[1] == 1, f'Unexpected nr of columns. Found {self.data.shape[1]} columns but expected 1'
        assert self.data.shape[0] > 0, f'No rows detected'
        
        # If bottom row contains escape character, remove.
        assert '\\' not in WD.data.iloc[-1, 0], 'Escape character found in bottom row'
        
    def extract_ward_ID(self):
        # Extract the last row as the ward ID
        #print(self.data.iloc[-1, 0])
        #assert bool(re.match(r"[A-z]", self.data.iloc[-1, 0])) == True, "Can't find characters in last row of dataframe"
        self.ward_ID = self.data.iloc[-1, 0]
        # Clean up trailing white space
        self.ward_ID = self.ward_ID.strip()
        
        self.data = self.data[:-1]
    
    def extract_candidates(self):
        self.candidates = list(self.data[self.data[0].str.contains('[A-z]', na = False)][0])
        
        # Account for inconsistent formatting: either ["candidate_name" "Party_name"] or [candidate_name "Party_name"]
        try:
            self.parties = [x.split('" "')[1].replace('"', '') for x in self.candidates]
        except:
            self.parties = [x.split('"')[1].replace('"', '') for x in self.candidates]
            
        self.parties = [x if x != '' else 'Independent' for x in self.parties]
        self.data = self.data[~self.data[0].str.contains('[A-z]', na = False)]
        
    def split_votes(self):
        # Remove 'end of data' indicator if any
        if self.data.iloc[-1,0] == '0':
            self.data = self.data[:-1]
            
        # Remove trailing zeroes and whitespace if any    
        self.data[0] = self.data[0].str.replace(' 0', '')
        self.data[0] = self.data[0].str.strip()
        # Remove extra whitespace between entries
        self.data[0] = self.data[0].str.replace(r'[ ]{2,}', ' ')

        
        # Split vote counts
        self.split_votes = self.data[0].str.split('\s', expand = True)
        
        # Rename vote count column
        self.split_votes = self.split_votes.rename(columns = {0:'Count'})
        self.counts = self.split_votes['Count']
        self.split_votes.drop('Count', axis = 1, inplace = True)
    
    def replace_parties(self):
        
        for x in range(len(self.parties)):
            self.split_votes.replace(str(x+1), self.parties[x], inplace = True)
            
    def combine(self):
        
        self.clean_data = self.split_votes.merge(self.counts, left_index=True, right_index=True)
        self.clean_data['Ward'] = self.ward_ID
        #print(self.clean_data)
        self.clean_data.dropna(subset=1)
        #print(self.clean_data[self.clean_data.isna()])
        self.clean_data.insert(0, 'Count', self.clean_data.pop('Count').dropna().astype('int'))
        self.clean_data.insert(0, 'Ward', self.clean_data.pop('Ward'))
        self.clean_data.dropna(subset='Count', inplace = True)
        
    def sort_by_count(self):
        self.clean_data = self.clean_data.sort_values(by='Count', ascending = False)
        

In [4]:
class Ward_Data_txt(Ward_Data):
    def read_data(self):
        # Check if top row contains text
        
        # Read in data
        self.data = pd.read_csv(self.path, header = None, delimiter = ';')
        
        assert self.data.shape[1] == 1, f'Unexpected nr of columns. Found {self.data.shape[1]} columns but expected 1'
        assert self.data.shape[0] > 0, f'No rows detected'

        
        # If bottom row contains escape character, remove.
        if self.data.iloc[-1, 0] == '\x0c':
            self.data = self.data.iloc[:-1, :]

In [5]:
class Ward_Data_csv(Ward_Data):
    
    def read_data(self):
        # Check if top row contains text
        
        # Read in data
        self.data = pd.read_csv(self.path, header = None, delimiter = ';')
        #print(self.data.shape)
        
        assert self.data.shape[1] == 1, f'Unexpected nr of columns. Found {self.data.shape[1]} columns but expected 1'
        assert self.data.shape[0] > 0, f'No rows detected'
        
        # If bottom row contains escape character, remove.
        assert '\\' not in self.data.iloc[-1, 0], 'Escape character found in bottom row'

In [6]:
class Ward_Data_xlsx(Ward_Data):
    
    def read_data(self):
        # Check if top row contains text
        
        # Read in data
        self.data = pd.read_excel(self.path, header = None)
        
        assert self.data.shape[1] == 1, f'Unexpected nr of columns. Found {self.data.shape[1]} columns but expected 1'
        assert self.data.shape[0] > 0, f'No rows detected'
        
        # If bottom row contains escape character, remove.
        assert '\\' not in self.data.iloc[-1, 0], 'Escape character found in bottom row'

## Main program function

In [7]:
def main():

    master_data = pd.DataFrame()
    completed_records = []
    nr_files = len(os.listdir('../_data/'))
    process_count = 0
    duplicates = []

    for file in glob.glob('../_data/*'):
        if process_count % 10 == 0:
            print(f'Processed {process_count} / {nr_files} files')
        WD = create_object(file)
        WD.read_data()
        WD.extract_ward_ID()

        if WD.ward_ID in completed_records:
            duplicates.append(WD.path)
            print('Ward already processed')
            continue
        else:
            completed_records.append(WD.ward_ID)

        

        WD.extract_candidates()
        WD.split_votes()
        WD.replace_parties()
        WD.combine()
        WD.sort_by_count()
        master_data = pd.concat([master_data, WD.clean_data])
        
        process_count += 1
    
    print(f'{process_count} files processed. {len(duplicates)} duplicate wards identified')
    
    return master_data

## Run the main function

In [9]:
master_data = main()
master_data.reset_index(drop=True, inplace = True)

Processed 0 / 319 files
Processed 10 / 319 files
Processed 20 / 319 files
Processed 30 / 319 files
Processed 40 / 319 files
Processed 50 / 319 files
Processed 60 / 319 files
Processed 70 / 319 files
Processed 80 / 319 files
Processed 90 / 319 files
Ward already processed
Processed 100 / 319 files
Processed 110 / 319 files
Ward already processed
Processed 120 / 319 files
Processed 130 / 319 files
Processed 140 / 319 files
Ward already processed
Processed 150 / 319 files
Processed 160 / 319 files
Processed 170 / 319 files
Ward already processed
Processed 170 / 319 files
Ward already processed
Processed 170 / 319 files
Processed 180 / 319 files
Processed 190 / 319 files
Processed 200 / 319 files
Processed 210 / 319 files
Ward already processed
Processed 220 / 319 files
Processed 230 / 319 files
Ward already processed
Processed 240 / 319 files
Ward already processed
Processed 250 / 319 files
Processed 260 / 319 files
Processed 270 / 319 files
Processed 280 / 319 files
Processed 290 / 319 f

In [10]:
master_data.head()

,Ward,Count,1,2,3,4,5,6,7,8,9,10,11,12,13
0,Ward 5 - Heldon and Laich,412.0,Scottish National Party (SNP),None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ward 5 - Heldon and Laich,333.0,Scottish Conservative and Unionist,Scottish Conservative and Unionist,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ward 5 - Heldon and Laich,217.0,Scottish Conservative and Unionist,Scottish Conservative and Unionist,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ward 5 - Heldon and Laich,173.0,Scottish Conservative and Unionist,Scottish Conservative and Unionist,Independent,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ward 5 - Heldon and Laich,142.0,Independent,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
master_data.to_csv('../_output/scottish_elections.csv')